In [14]:
#Impory the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import pickle

In [15]:
#load data
originalDataset = pd.read_csv('news_articles.csv')
textData = originalDataset.iloc[:,10]
fakeOrReal = originalDataset.iloc[:,8]
fakeOrReal = fakeOrReal.replace(['Real'],1)
fakeOrReal = fakeOrReal.replace(['Fake'],0)
dataset = pd.concat([textData, fakeOrReal], axis=1, sort=False)
dataset.drop(dataset.tail(50).index,inplace=True)

In [24]:
print(dataset)

                                 text_without_stopwords  label
0     print pay back money plus interest entire fami...    1.0
1     attorney general loretta lynch plead fifth bar...    1.0
2     red state fox news sunday reported morning ant...    1.0
3     email kayla mueller prisoner tortured isis cha...    1.0
4     email healthcare reform make america great sin...    1.0
...                                                 ...    ...
2041  check hillarythemed haunted house anticlinton ...    1.0
2042  good samaritan wearing indian headdress disarm...    1.0
2043  skype sex scam fortune built shame moroccan bo...    1.0
2044  posted eddie skyhigh potency may scare away cr...    1.0
2045  billion even known keeping supposedly deleted ...    1.0

[2046 rows x 2 columns]


In [16]:
#creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(dataset.iloc[:,0]).toarray()
y = dataset.iloc[:,-1]

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 1)

In [18]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', max_depth = 75)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=75, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[256,  17],
       [ 65,  72]], dtype=int64)

In [21]:
#Save file
filename = 'modelXX_randomForest.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [22]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8


In [23]:
#move threshold
threshold = 0.425

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

cm = confusion_matrix(y_test, predicted)
cm

array([[228,  45],
       [ 41,  96]], dtype=int64)